In [2]:
import radarsimpy
print('`RadarSimPy` used in this example is version: ' +
      str(radarsimpy.__version__))


`RadarSimPy` used in this example is version: 11.2.0


# FMCW Radar DoA Estimation

[![Documentations](https://img.shields.io/github/v/tag/radarsimx/radarsimpy?label=Documentation&logo=read-the-docs)](https://radarsimx.github.io/radarsimpy/)
[![Download](https://img.shields.io/github/v/tag/radarsimx/radarsimpy?label=Download&logo=python)](https://radarsimx.com/product/radarsimpy/)

## Introduction

Direction of Arrival (DoA) estimation is a fundamental technique used in radar systems to determine the direction from which a signal or target is emanating. This technique is particularly valuable in applications like radar surveillance, target tracking, navigation, and communication systems. By accurately estimating the direction of arrival of incoming signals, radar systems can effectively locate and track objects of interest.

DoA estimation involves determining the angles at which signals impinge upon an array of receiving antennas. This can be accomplished through various methods, each with its own advantages and limitations. Some common techniques for DoA estimation include:

1. **Beamforming:** Beamforming involves combining signals received from different antennas in a way that enhances the desired signal coming from a specific direction while suppressing interference from other directions. This can provide a rough estimate of the DoA based on which antenna receives the strongest signal.

2. **MUSIC (MUltiple SIgnal Classification):** The MUSIC algorithm exploits the signal subspace and noise subspace of the received signals to estimate the DoA with high resolution. It identifies the angles corresponding to the peaks in the spatial spectrum, providing accurate localization even for closely spaced signals.

3. **ESPRIT (Estimation of Signal Parameters via Rotational Invariance Techniques):** ESPRIT is a high-resolution DoA estimation technique that relies on exploiting the rotational invariance property of the signal subspace. It is computationally efficient and provides precise estimates of DoA for both coherent and incoherent signals.

4. **Capon's Method:** Capon's Minimum Variance Distortionless Response (MVDR) beamformer minimizes the output power subject to a constraint on the output power at the desired direction. It offers improved performance in the presence of spatially colored noise and interference.

5. **Phase Comparison Monopulse:** This method uses the phase differences between the signals received by different antennas to determine the DoA. It is often used in tracking and navigation systems.

6. **Correlation Techniques:** Cross-correlation between the signals received by different antennas can provide an estimate of the phase difference, which corresponds to the DoA.

7. **Coherent Processing:** This involves maintaining phase coherence between transmitted and received signals to achieve accurate DoA estimation. Coherent processing requires synchronization between the transmitter and receiver.

DoA estimation plays a crucial role in enhancing the capabilities of radar systems, enabling them to accurately locate and track multiple targets, discriminate between closely spaced targets, and improve overall situational awareness. The choice of DoA estimation technique depends on factors like the accuracy required, computational complexity, noise environment, and system constraints.

This is an example of using DoA algorithms (MUSIC & Esprit) with [`RadarSimPy`](https://radarsimx.com/radarsimx/radarsimpy/).



## Setup FMCW radar

### Transmitter

Setup the basic transmitter parameters through `Transmitter` module.

### Receiver

Setup the receiver parameters through `Receiver` module.


In [3]:
import numpy as np
from radarsimpy import Radar, Transmitter, Receiver
wavelength = 3e8 / 60.5e9

channels = []

N_tx = 2
N_rx = 64

channels.append(
    dict(
        location=(0, -N_rx/2*wavelength/2, 0),
    ))

channels.append(
    dict(
        location=(0, wavelength*N_rx/2-N_rx/2*wavelength/2, 0),
    ))

tx = Transmitter(f=[61e9, 60e9],
                 t=[0, 16e-6],
                 tx_power=15,
                 prp=40e-6,
                 pulses=512,
                 channels=channels)

channels = []
for idx in range(0, N_rx):
    channels.append(
        dict(
            location=(0, wavelength/2*idx-(N_rx-1) * wavelength/4, 0),
        ))

rx = Receiver(fs=20e6,
              noise_figure=8,
              rf_gain=20,
              load_resistor=500,
              baseband_gain=30,
              channels=channels)

radar = Radar(transmitter=tx, receiver=rx)


There are 2 transmitters and 64 receivers, the synthesized array is a 1 x 128 uniform linear array.

> Note: The modulation is not considered here. Assume the 2 transmitters are orthogonal.

In [4]:
import plotly.graph_objs as go
from IPython.display import Image

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=radar.transmitter.locations[:, 1]/wavelength,
               y=radar.transmitter.locations[:, 2]/wavelength,
               mode='markers',
               name='Transmitter',
               opacity=0.7,
               marker=dict(size=10)))

fig.add_trace(
    go.Scatter(x=radar.receiver.locations[:, 1]/wavelength,
               y=radar.receiver.locations[:, 2]/wavelength,
               mode='markers',
               opacity=1,
               name='Receiver'))

fig.update_layout(
    title='Array configuration',
    xaxis=dict(title='y (λ)'),
    yaxis=dict(title='z (λ)',
               scaleanchor="x",
               scaleratio=1),
)

fig.show()


## Targets

Create 3 targets at 40 m. The azimuth angles of the 3 targets are -5, -4, and 45 degrees relative to the radar.

In [5]:
true_theta = [-5, -4, 45]

target_1 = dict(location=(40*np.cos(np.radians(true_theta[0])), 40*np.sin(np.radians(true_theta[0])),
                0), speed=(0, 0, 0), rcs=10, phase=0)
target_2 = dict(location=(40*np.cos(np.radians(true_theta[1])), 40*np.sin(np.radians(true_theta[1])),
                0), speed=(0, 0, 0), rcs=10, phase=0)
target_3 = dict(location=(40*np.cos(np.radians(true_theta[2])), 40*np.sin(np.radians(true_theta[2])),
                0), speed=(0, 0, 0), rcs=10, phase=0)

targets = [target_1, target_2, target_3]


## Simulate Baseband Signals
 
Use the `simulator` module to simulate the baseband samples. The user can choose between Python engine `simpy` or C++ engine `simc`.

The output baseband data is a 3-D matrix:

$[channels, pulses, ADC~samples]$

In [6]:
from radarsimpy.simulator import simc

data = simc(radar, targets, noise=True)
timestamp = data['timestamp']
baseband = data['baseband']


## Radar signal processing

### Range-Doppler processing

In [7]:
from scipy import signal
import radarsimpy.processing as proc

range_window = signal.chebwin(radar.samples_per_pulse, at=80)
doppler_window = signal.chebwin(radar.transmitter.pulses, at=60)

range_doppler = proc.range_doppler_fft(
    baseband, rwin=range_window, dwin=doppler_window)


C:\Users\zjx8rj\AppData\Local\Temp\ipykernel_38892\3038848528.py:4: DeprecationWarning:

Importing chebwin from 'scipy.signal' is deprecated and will raise an error in SciPy 1.13.0. Please use 'scipy.signal.windows.chebwin' or the convenience function 'scipy.signal.get_window' instead.

C:\Users\zjx8rj\AppData\Local\Temp\ipykernel_38892\3038848528.py:5: DeprecationWarning:

Importing chebwin from 'scipy.signal' is deprecated and will raise an error in SciPy 1.13.0. Please use 'scipy.signal.windows.chebwin' or the convenience function 'scipy.signal.get_window' instead.



Find the beam vector of the peak and create the covariance matrix.

In [8]:
from scipy import linalg

det_idx = [np.argmax(np.mean(np.abs(range_doppler[:, 0, :]), axis=0))]

bv = range_doppler[:, 0, det_idx[0]]
bv = bv/linalg.norm(bv)

snapshots = 20

bv_snapshot = np.zeros((N_tx*N_rx-snapshots, snapshots), dtype=complex)

for idx in range(0, snapshots):
    bv_snapshot[:, idx] = bv[idx:(idx+N_tx*N_rx-snapshots)]

covmat = np.cov(bv_snapshot.conjugate())


FFT of the beam vector. Two peaks can be seen, and it is not able to discriminate the targets at -5 and -4 degrees.

In [9]:
from scipy import fft
from scipy import signal

fft_spec = 20 * \
    np.log10(np.abs(fft.fftshift(fft.fft(bv.conjugate(), n=1024))))

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arcsin(np.linspace(-1, 1, 1024, endpoint=False))/np.pi*180,
                         y=fft_spec,
                         name='FFT')
              )

fig.update_layout(
    title='FFT',
    yaxis=dict(title='Amplitude (dB)'),
    xaxis=dict(title='Angle (deg)'),
    margin=dict(l=10, r=10, b=10, t=40),
)

fig.show()


### MUSIC

In [10]:
from radarsimpy.processing import doa_music

scan_angle = np.arange(-90, 90, 0.1)
music_doa, music_idx, ps_db = doa_music(covmat, 3, scanangles=scan_angle)

print('DoAs from MUSIC:', music_doa, 'degrees')


DoAs from MUSIC: [-4.  -5.  45.1] degrees


In [11]:
import matplotlib.pyplot as plt

fig = go.Figure()

fig.add_trace(go.Scatter(x=scan_angle,
                         y=ps_db,
                         name='Pseudo Spectrum')
              )

fig.add_trace(go.Scatter(x=music_doa,
                         y=ps_db[music_idx],
                         mode='markers',
                         name='Estimated DoA')
              )
fig.update_layout(
    title='MUSIC',
    yaxis=dict(title='Amplitude (dB)'),
    xaxis=dict(title='Angle (deg)'),
    margin=dict(l=10, r=10, b=10, t=40),
)
fig.show()

### Root-MUSIC

In [12]:
from radarsimpy.processing import doa_root_music

rootmusic_doa = doa_root_music(covmat, 3)
print('DoAs from Root-MUSIC:', rootmusic_doa, 'degrees')


DoAs from Root-MUSIC: [45.05086294 -4.99488101 -4.00811728] degrees


### ESPRIT

In [13]:
from radarsimpy.processing import doa_esprit

esprit_doa = doa_esprit(covmat, 3)
print('DoAs from ESPRIT:', esprit_doa, 'degrees')


DoAs from ESPRIT: [45.05540944 -4.00570542 -4.99967713] degrees


### Capon beamformer

In [14]:
from radarsimpy.processing import doa_capon

ps_capon = doa_capon(covmat, scanangles=scan_angle)


In [15]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=scan_angle,
                         y=ps_capon,
                         name='Pseudo Spectrum')
              )

fig.update_layout(
    title='Capon',
    yaxis=dict(title='Amplitude (dB)'),
    xaxis=dict(title='Angle (deg)'),
    margin=dict(l=10, r=10, b=10, t=40),
)
fig.show()

### Bartlett beamformer

In [16]:
from radarsimpy.processing import doa_bartlett

ps_bartlett = doa_bartlett(covmat, scanangles=scan_angle)


In [17]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=scan_angle,
                         y=ps_bartlett,
                         name='Pseudo Spectrum')
              )

fig.update_layout(
    title='Bartlett',
    yaxis=dict(title='Amplitude (dB)'),
    xaxis=dict(title='Angle (deg)'),
    margin=dict(l=10, r=10, b=10, t=40),
)
fig.show()